&nbsp;

# 15. Pré-traitement numérique pour SciML

---

&nbsp;

## 1. Introduction

Ce notebook vise à préparer l’état réduit issu de l’analyse EOF afin qu’il soit compatible avec l’apprentissage de modèles dynamiques SciML, cela en garantissant la stabilité numérique, l’absence de fuite d’information et la cohérence temporelle. On peut le réduire à une question centrale.

&nbsp;

> #### Est-ce mon état réduit est numériquement sain pour apprendre une dynamique ?

L’état réduit est constitué des $K$ premiers coefficients EOF (~150 modes dans le NetCDF), stockés sous la forme d’une série temporelle multivariée.

Ce que l'on connaît des modèles dynamique SciML, c'est leur tendance à être sensible aux échelles. Nous l'apprenons de cette source Application of Reduced-Order Models for Temporal Multiscale Representations in the Prediction of Dynamical Systems (https://arxiv.org/html/2510.18925v1) après une recherche avec le mot-clé "Model Reduction".

Globalement, ce que l'on en retire, c'est que ce type de modèle est conçu pour apprendre une loi d'évolution temporelle. Si l'état réduit "contient" des composantes associées à diverses échelles temporelles, le modèle appliqué doit réussir à différencier ces échelles au risque de sur-lisser ou ne modèliser qu'une seule échelle.

Dans un système réel (comme la SST), il existe des interactions entre échelles lentes et rapides (ou un spectre de K échelles). Les approches dites "naïves" peinent à capturer ces comportements qui opèrent simultanément sur le même jeu de données. 

Chaque mode (ou degré de liberté) peut donc évoluer à son propre rythme, et dans un système tel à plusieurs échelles, il nous faut au plus simplifier ce travail de différenciation pour que la formulation proposée par la modélisation dynamique SciML reproduise au plus fidèlement les signatures lentes/rapides et leurs couplages(ou le fait que l'évolution d’une composante du système dépend de l’état d’une ou plusieurs autres composantes). Exemple : un gradient thermique large échelle (EOF 1) peut conditionner l’intensité des fronts côtiers (EOF 3).

&nbsp;

---

&nbsp;

## 2. Normalisation et mise à l'échelle

> #### Pourquoi faire cela ?

La normalisation et la mise à l'échelle sont une réponse simple et efficace au problème de différencition d'échelles exprimée ci-dessus. On veut éviter qu'un mode ne domine trop numériquement. Mathématiquement cela se traduit par une normalisation par l'écart-type (ou scaling), comme suit :

$$
\tilde{a}_k(t) = \frac{a_k(t)}{\sigma_k}
$$
où :
- $\tilde{a}_k(t)$ : état réduit $k$ normalisé de sortie
- $a_k(t)$ : état réduit $k$ d'entrée 
- $\sigma_k$ : écart-type des PCs

En effet, diviser par l'écart-type ajuste notre état réduit de manière à affaiblir les PCs qui dominent et renforcer ceux qui sont dominés.

&nbsp;

In [6]:
import xarray as xr
import numpy as np

dsReducedState = xr.open_dataset("data/processed/sstReducedStateCOPERNICUS20102019.nc")

PCs = dsReducedState["PCs"].values

# We recheck if PCs are welled centered
print(PCs.mean(axis=0))  # Should be close to zero

Nt = PCs.shape[0]
trainRatio = 0.8
trainEnd = int(trainRatio * Nt)

# Temporal splitting
PCsTrain = PCs[:trainEnd]
PCsVal   = PCs[trainEnd:]

# Normalization only on training data so we avoid data leakage (lookahead bias)
meanTrain = PCsTrain.mean(axis=0)
stdTrain  = PCsTrain.std(axis=0)
PCsTrainScaled = (PCsTrain - meanTrain) / stdTrain
PCsValScaled   = (PCsVal   - meanTrain) / stdTrain


[ 1.23361372e-06  5.43166081e-08  1.03606318e-07 -5.01841100e-07
  1.06021844e-07  2.24186579e-07 -2.36851747e-07  1.96359110e-07
  1.07066391e-08  7.57298864e-08 -1.63406824e-07 -2.90025888e-08
  6.17590317e-08  1.07066391e-08  2.18572126e-07  1.99182665e-07
 -1.27663611e-07  8.63508447e-08 -9.18714562e-08 -6.73612490e-08
 -5.44798198e-08  8.75055619e-08  4.72495749e-08  3.06020560e-08
 -4.69394728e-08  3.78649423e-09  2.03483275e-08  8.17360544e-08
 -1.51786193e-08 -1.24693173e-07  5.24478452e-08  6.78304772e-08
  2.88556983e-08  9.12675731e-08 -1.70392251e-08  3.54494212e-08
 -2.70277365e-08 -3.44048701e-08  5.81031045e-09 -1.14492494e-07
 -5.69606273e-09 -2.30780302e-08 -1.44931338e-08 -1.56437707e-08
  6.02738126e-08  4.19452162e-09 -4.52747209e-08 -5.74502579e-09
  2.79417165e-08 -1.03149329e-08  3.84198628e-08  4.18636148e-09
  3.71794577e-08  1.53907944e-08 -4.21084287e-08 -3.00716216e-08
 -4.14882280e-08  9.77633690e-08 -1.80837745e-08 -6.29846397e-08
  2.38288012e-09 -2.69624

&nbsp;

On revérifie que le centrage est correct pour garantir l'absence de biais constant dans la dynamique que l'on souhaite apprendre. On constate la moyenne de chaque PC est proche de 0 (de l'ordre du dix-millième au cent-milliardième selon le PC).